In [1]:
import os
from datetime import datetime
import sys
import cdsapi
import copernicusmarine as cm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from neural_lam import constants



c:\Users\Gabri\anaconda3\envs\tfg_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
static_dir = os.path.join(BASE_DIR, "data", "atlantic", "static")
os.makedirs(static_dir, exist_ok=True)

### Ajustamos la función para cargar la máscara

In [4]:
def load_mask(path):
    """
    Load bathymetry mask.

    Args:
    path (str): Path to the bathymetry mask file.

    Returns:
    mask (xarray.Dataset): Bathymetry mask.
    """
    if os.path.exists(path):
        print("Bathymetry mask file found. Loading from file.")
        mask = xr.load_dataset(path).mask
    else:
        print("Bathymetry mask file not found. Downloading...")
        bathy_data = cm.open_dataset(
            dataset_id="cmems-IFREMER-ATL-SST-L4-REP-OBS_FULL_TIME_SERIE",
            variables=["analysed_sst"],
            minimum_longitude=-20.97,
            maximum_longitude=-5.975,
            minimum_latitude=19.55,
            maximum_latitude=34.525,
            start_datetime="2023-12-30T00:00:00",
            end_datetime="2023-12-31T00:00:00",
        )
        sst = bathy_data["analysed_sst"]
        mask = xr.where(np.isnan(sst.isel(time=0)), 0, 1)
        mask.name = "mask"

        out_path = os.path.join(static_dir, "bathy_mask.nc")
        mask.to_netcdf(out_path)
        print(f"mask saved in: {out_path}")

    return mask

### Creamos una máscara para el mar siguiendo la estructura del proyecto original

In [5]:
def create_sea_mask(path_prefix):
    mask_data = load_mask(os.path.join(static_dir, "bathy_mask.nc"))

    sea_mask = (mask_data.values == 1)
    
    out_path = os.path.join(path_prefix, "sea_mask.npy")
    np.save(out_path, sea_mask)
    print(f"Sea mask saved in: {out_path}")

### Comprobamos que la área tomada es la correcta

In [ ]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
static_dir = os.path.join(BASE_DIR, "data", "atlantic", "static")
os.makedirs(static_dir, exist_ok=True)
mask = load_mask(os.path.join(static_dir, "bathy_mask.nc"))
create_sea_mask(static_dir)
plt.figure(figsize=(8, 6))
plt.imshow(mask.values, origin='lower', cmap='viridis')
plt.title("Desired Mask")
plt.colorbar()
plt.show()

NameError: name 'os' is not defined

In [10]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
static_dir = os.path.join(BASE_DIR, "data", "atlantic", "static")
os.makedirs(static_dir, exist_ok=True)

In [3]:
def select(dataset, mask):
    """
    Select masked volume.

    Args:
    dataset (xarray.Dataset): Input dataset.
    mask (xarray.Dataset): Bathymetry mask.

    Returns:
    mask (xarray.Dataset): Masked dataset.
    """
    # Fix longitude mismatches close to zero
    dataset["longitude"] = dataset.longitude.where(
        (dataset.longitude < -1e-9) | (dataset.longitude > 1e-9), other=0.0
    )

    #Select depth levels, and masked area
    if hasattr(dataset, "depth"):
        dataset = dataset.sel(depth=constants.DEPTHS)
        dataset = dataset.where(mask, drop=True)

    # Uncomment to coarsen grid
    # dataset = dataset.coarsen(longitude=2, boundary="pad").mean()
    # dataset = dataset.coarsen(latitude=2, boundary="pad").mean()
    return dataset

### Ajustamos la función de descarga de los datos de era5

In [4]:
def download_era5(
    start_date,
    end_date,
    request_variables,
    ds_variables,
    static_path,
    path_prefix,
    mask,
):
    """
    Download and save daily ERA5 data.

    Args:
    start_date (datetime): The start date for data retrieval.
    end_date (datetime): The end date for data retrieval.
    request_variables (list): List of variables to request from cds.
    ds_variables (list): List of variables in the dataset.
    static_path (str): Location of static data.
    path_prefix (str): The directory path prefix where the files will be saved.
    mask (xarray.Dataset): Bathymetry mask.
    """
    grid_mask = np.load(f"{static_path}/sea_mask.npy")[0]
    print(grid_mask.shape)
    print(mask.latitude.shape)
    print(mask.longitude.shape)
    client = cdsapi.Client()
    current_date = start_date
    while current_date <= end_date:
        year = current_date.year
        month = current_date.month

        filename = f"{path_prefix}/{current_date.strftime('%Y%m')}.nc"
        if os.path.isfile(filename):
            if month == 7:
                current_date = current_date.replace(
                    year=year + 1, month=1, day=1
                )
            else:
                current_date = current_date.replace(month=month + 6, day=1)
            continue

        client.retrieve(
            "reanalysis-era5-single-levels",
            {
                "format": "netcdf",
                "product_type": ["reanalysis"],
                "variable": request_variables,
                "year": [str(year)],
                "month": [f"{m:02d}" for m in range(month, month + 6)],
                "day": [f"{d:02d}" for d in range(1, 32)],
                "time": [f"{h:02d}:00" for h in range(0, 24, 6)],
                "area": [
                    mask.latitude.max().item(),
                    mask.longitude.min().item(),
                    mask.latitude.min().item(),
                    mask.longitude.max().item(),
                ],
                "download_format": "unarchived",
            },
            filename,
        )
        print(f"Downloaded {filename}")

        # Load the data and average to daily
        ds = xr.open_dataset(filename)
        daily_ds = ds.resample(valid_time="1D").mean()

        # Interpolate to the bathymetry mask grid
        interp_daily_ds = daily_ds.interp(
            longitude=mask.longitude, latitude=mask.latitude
        )

        # Apply the bathymetry mask to select the exact area
        masked_data = select(interp_daily_ds, mask)

        # Save in numpy format with shape (n_grid, f)
        for single_date in masked_data.valid_time.values:
            date_str = pd.to_datetime(single_date).strftime("%Y%m%d")
            daily_data = masked_data.sel(valid_time=single_date)
            combined_data = []
            for var in ds_variables:
                data = daily_data[var].values  # h, w
                combined_data.append(data)

            combined_data = np.stack(combined_data, axis=-1)  # h, w, f
            clean_data = np.nan_to_num(combined_data, nan=0.0)
            
            np.save(
                f"{path_prefix}/{date_str}.npy", clean_data[grid_mask, :]
            )  # n_grid, f
            print(f"Saved daily data to {path_prefix}/{date_str}.npy")

        # Increment to the next month
        if month == 7:
            current_date = current_date.replace(year=year + 1, month=1, day=1)
        else:
            current_date = current_date.replace(month=month + 6, day=1)

### Ajustamos los parámetros que se le pasan a la función de descarga

In [5]:
start_date = datetime.strptime("2024-05-30", "%Y-%m-%d")
end_date = datetime.strptime("2024-05-31", "%Y-%m-%d")
request_variables = [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind"
        ]
ds_variables = ["u10", "v10"]
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

static_path = os.path.join(base_path, "data", "atlantic", "static")
raw_path = os.path.join(base_path, "data", "atlantic", "raw")

os.makedirs(static_path, exist_ok=True)
os.makedirs(raw_path, exist_ok=True)

era5_path = os.path.join(raw_path, "era5")
os.makedirs(era5_path, exist_ok=True)

bathymetry_mask_path = os.path.join(static_path, "bathy_mask.nc")

print(bathymetry_mask_path)
mask = load_mask(bathymetry_mask_path)
download_era5(start_date=start_date,
              end_date = end_date,
              request_variables=request_variables, 
              ds_variables=ds_variables,
              static_path=static_path,
              path_prefix=era5_path,
              mask = mask
              )


d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\static\bathy_mask.nc
Bathymetry mask file found. Loading from file.
(300, 300)
(300,)
(300,)


2025-02-14 12:43:42,028 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-14 12:43:42,029 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-14 12:43:43,769 INFO Request ID is ae9480d0-3d74-4410-bf5d-a58263556684
2025-02-14 12:43:43,875 INFO status has been updated to accepted
2025-02-14 12:43:58,953 INFO status has been updated to successful


Downloaded d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/202405.nc
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240501.npy
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240502.npy
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240503.npy
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240504.npy
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240505.npy
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240506.npy
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240507.npy
Saved daily data to d:\ULPGC\4Curso\TFG\seacast_code\Proyecto_TFG\Seacast\data\atlantic\raw\era5/20240508.npy
Saved daily data to d:

### Comprobamos que las dimensiones de los datos cargados son correctos

In [6]:
era5_file = os.path.join(project_root, "data", "atlantic", "raw", "era5", "20241031.npy")
static_dir = os.path.join(project_root, "data", "atlantic", "static")
sea_mask_file = os.path.join(static_dir, "sea_mask.npy")
mask_nc_file = os.path.join(static_dir, "bathy_mask.nc")
era5_data = np.load(era5_file)

print("shape of era5 data downloaded: ", era5_data.shape)
sea_mask = np.load(sea_mask_file)
grid_mask = sea_mask.astype(bool)

n_grid_expected = np.sum(grid_mask)
print("n_grid expected: ", n_grid_expected)
print(sea_mask.shape)

shape of era5 data downloaded:  (49061, 2)
n_grid expected:  49061
(1, 300, 300)
